Goal: separate the ingredients in the FoodKG table into a list of ingredients

In [1]:
import json
import os
import time

import pandas as pd
from dotenv import load_dotenv

from llm_util import process_food_kg_df, assemble_food_kg_df

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key,
    timeout=300,
)


In [2]:
df = pd.read_csv("data/raw/nourish_public_FoodKG.csv")

In [3]:
df.head(5)

,id,title,ingredients,directions,link,source,ner
0,2229479,Double Chocolate Chewy Cookies,"['3/4 cup softened salted butter', '1 1/2 cup ...","['Heat oven to 350F.', 'Cream butter and sugar...",cookpad.com/us/recipes/339167-double-chocolate...,Recipes1M,"['butter', 'sugar', 'eggs', 'vanilla', 'flour'..."
1,2229480,"""Old Fashioned Potato Soup - ""Homemade"" Potato...","['6 Potatoes, peeled, cubes', '2 Leeks, washed...",['Put all ingredients except milk and chives i...,online-cookbook.com/goto/cook/rpage/0003A6,Recipes1M,"['Potatoes', 'Leeks', 'Onions', 'Ham bone', 'C..."
2,2229492,Persian pasta with spiced chickpeas and pistac...,"['250 g (8.8oz) spaghetti', '1 tin of chickpea...","['Boil a large pot of water for your pasta.', ...",www.lovefood.com/guide/recipes/17869/persian-p...,Recipes1M,"['chickpeas', 'onion', 'tomatoes', 'clove of g..."
3,2229494,Shawn & Lauras Kitchen Sink Feijoada,"['1 12 tablespoons garlic (chopped)', '1 shall...","['Chop garlic, shallot, green onions, parsley ...",www.food.com/recipe/shawn-lauras-kitchen-sink-...,Recipes1M,"['garlic', 'shallot', 'Italian parsley', 'oreg..."
4,2229498,The 4:2:1 Golden Ratio for Always Delicious Ta...,"['4 Eggs (large)', '2 tbsp Sugar', '1 tsp Salt...","['Break the eggs into a bowl.', 'Put all the i...",cookpad.com/us/recipes/145525-the-421-golden-r...,Recipes1M,"['Eggs', 'Sugar', 'Salt', 'Mayonnaise']"


Performance:

model = "qwen/qwen3-4b-2507"
* Overlap:   0.8744 (Intersection divided by the size of the larger set)
* Jaccard:   0.8104 (Intersection divided by the size of the union)
* Precision: 0.8834 (Fraction of predicted ingredients that are correct)
* Recall:    0.8756 (Fraction of actual ingredients that were predicted)
* F1:        0.8791 (Harmonic mean of precision and recall)


model = "qwen/qwen3-4b-thinking-2507"
* Took way too long to even consider

In [4]:
model = "qwen/qwen3-4b-2507"
# model="openai/gpt-oss-20b",

start_time = time.time()

# there is a bug with the checkpoints it ain't working and request times out
process_food_kg_df(
    df=df,
    model=model,
    client=client,
    # restart=True,  # reset the checkpoints
    batch_size=1000,
    # stop_at=100
)

end_time = time.time()
print(f"{model} completed in {end_time - start_time:.2f} seconds")

APITimeoutError: Request timed out.

In [ ]:
end_time = time.time()
print(f"{model} completed in {end_time - start_time:.2f} seconds")

In [ ]:
final_df = assemble_food_kg_df()

In [ ]:
len(final_df)

In [ ]:
# CSV can’t store native Python lists (they are objects), so use JSON dump
ingredients_table = pd.DataFrame({
    "id": final_df["id"],
    "title": final_df["title"],
    "ingredients": final_df["ingredients"],  # new, kept upon request
    "ingredients_normalized": final_df["ingredients_normalized"].apply(json.dumps),
}).sort_values("id")
# Create a directory if it doesn't exist and save

In [ ]:
ingredients_table.head(5)

In [ ]:
os.makedirs("data/output", exist_ok=True)
ingredients_table.to_csv("data/output/processed_ingredients.csv", index=False)

In [ ]:
final_df.head()